In [1]:
import pandas as pd
import os
import sys
import math

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [5]:
# pull .xlsx from aws
enviroscreen_excel = 's3://ca-climate-index/1_pull_data/society_economy/vulnerable_populations/ca_enviro_screen/calenviroscreen.xlsx'
enviroscreen_data = pd.read_excel(enviroscreen_excel)

In [60]:
enviroscreen_data

,Census Tract,Total Population,California County,ZIP,Approximate Location,Longitude,Latitude,CES 4.0 Score,CES 4.0 Percentile,CES 4.0 Percentile Range,...,Linguistic Isolation Pctl,Poverty,Poverty Pctl,Unemployment,Unemployment Pctl,Housing Burden,Housing Burden Pctl,Pop. Char.,Pop. Char. Score,Pop. Char. Pctl
0,6019001100,2780,Fresno,93706,Fresno,-119.781696,36.709695,93.183570,100.000000,95-100% (highest scores),...,79.374746,76.0,98.919598,12.8,93.831338,30.3,91.039290,93.155109,9.663213,99.722642
1,6077000700,4680,San Joaquin,95206,Stockton,-121.287873,37.943173,86.653790,99.987393,95-100% (highest scores),...,95.533902,73.2,98.391960,19.8,99.206143,31.2,92.281369,93.165408,9.664281,99.735250
2,6037204920,2751,Los Angeles,90023,Los Angeles,-118.197497,34.017500,82.393909,99.974786,95-100% (highest scores),...,81.553661,62.6,93.391960,6.4,61.530453,20.3,63.967047,83.751814,8.687785,95.789208
3,6019000700,3664,Fresno,93706,Fresno,-119.827707,36.734535,81.327940,99.962179,95-100% (highest scores),...,78.711598,65.7,95.351759,15.7,97.345133,35.4,96.413181,94.641227,9.817371,99.886536
4,6019000200,2689,Fresno,93706,Fresno,-119.805504,36.735491,80.745476,99.949571,95-100% (highest scores),...,86.561104,72.7,98.304020,13.7,95.288912,32.7,94.157161,95.398873,9.895964,99.949571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8030,6107004000,582,Tulare,93257,Porterville,-118.983849,36.038061,NaN,NaN,NaN,...,NaN,79.6,99.422111,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8031,6109985202,2509,Tuolumne,95327,Unincorporated Tuolumne County area,-120.537071,37.891939,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8032,6111001206,778,Ventura,93001,Unincorporated Ventura County area,-119.371944,34.343903,NaN,NaN,NaN,...,NaN,17.1,27.349246,NaN,NaN,24.4,78.466413,NaN,NaN,NaN
8033,6111003012,675,Ventura,93036,Oxnard,-119.180105,34.235076,NaN,NaN,NaN,...,99.553390,96.7,100.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
enviroscreen_data.columns

Index(['Census Tract', 'Total Population', 'California County', 'ZIP',
       'Approximate Location', 'Longitude', 'Latitude', 'CES 4.0 Score',
       'CES 4.0 Percentile', 'CES 4.0 Percentile Range', 'Ozone', 'Ozone Pctl',
       'PM2.5', 'PM2.5 Pctl', 'Diesel PM', 'Diesel PM Pctl', 'Drinking Water',
       'Drinking Water Pctl', 'Lead', 'Lead Pctl', 'Pesticides',
       'Pesticides Pctl', 'Tox. Release', 'Tox. Release Pctl', 'Traffic',
       'Traffic Pctl', 'Cleanup Sites', 'Cleanup Sites Pctl',
       'Groundwater Threats', 'Groundwater Threats Pctl', 'Haz. Waste',
       'Haz. Waste Pctl', 'Imp. Water Bodies', 'Imp. Water Bodies Pctl',
       'Solid Waste', 'Solid Waste Pctl', 'Pollution Burden',
       'Pollution Burden Score', 'Pollution Burden Pctl', 'Asthma',
       'Asthma Pctl', 'Low Birth Weight', 'Low Birth Weight Pctl',
       'Cardiovascular Disease', 'Cardiovascular Disease Pctl', 'Education',
       'Education Pctl', 'Linguistic Isolation', 'Linguistic Isolation Pctl',

In [89]:
metric_enviroscreen_data = enviroscreen_data[['Census Tract', 
                                              'California County', 
                                              'Total Population', 
                                              'Drinking Water',
                                              'Drinking Water Pctl',
                                              'Groundwater Threats', 
                                              'Groundwater Threats Pctl', 
                                              'Asthma',
                                              'Asthma Pctl',
                                              'Low Birth Weight', 
                                              'Cardiovascular Disease', 
                                              'Education', 
                                              'Linguistic Isolation',
                                              'Poverty',
                                              'Unemployment', 
                                              'Housing Burden'
                                              ]]

In [90]:
metric_enviroscreen_data 

,Census Tract,California County,Total Population,Drinking Water,Drinking Water Pctl,Groundwater Threats,Groundwater Threats Pctl,Asthma,Asthma Pctl,Low Birth Weight,Cardiovascular Disease,Education,Linguistic Isolation,Poverty,Unemployment,Housing Burden
0,6019001100,Fresno,2780,733.946935,84.388660,54.25,91.167192,129.54,97.220837,7.80,21.47,44.5,16.0,76.0,12.8,30.3
1,6077000700,San Joaquin,4680,389.846569,41.551143,78.60,95.085506,105.88,94.217348,6.88,20.26,46.4,29.7,73.2,19.8,31.2
2,6037204920,Los Angeles,2751,787.940335,92.531535,20.50,68.919143,76.10,82.764207,7.11,20.87,52.2,17.1,62.6,6.4,20.3
3,6019000700,Fresno,3664,733.946935,84.388660,9.50,44.828159,139.45,98.242772,10.65,22.68,41.4,15.7,65.7,15.7,35.4
4,6019000200,Fresno,2689,733.946935,84.388660,28.25,78.050805,139.08,98.167996,10.25,22.64,43.6,20.0,72.7,13.7,32.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8030,6107004000,Tulare,582,634.315048,71.150244,0.00,0.000000,61.64,71.086740,NaN,21.22,43.6,NaN,79.6,NaN,NaN
8031,6109985202,Tuolumne,2509,393.187253,42.725116,3.00,22.115225,68.79,77.567298,NaN,22.89,34.1,NaN,NaN,NaN,NaN
8032,6111001206,Ventura,778,406.952403,44.810791,19.05,66.959987,48.36,54.062812,NaN,8.77,NaN,NaN,17.1,NaN,24.4
8033,6111003012,Ventura,675,663.986626,72.923692,0.30,2.108584,45.35,49.252243,NaN,12.25,NaN,51.9,96.7,NaN,NaN


In [50]:
est_pop = "s3://ca-climate-index/0_map_data/cri_acs_demographic_estimated_population.csv"
ca_est_pop = pd.read_csv(est_pop)
ca_est_pop = ca_est_pop[['Census_Tract', 'est_total_pop']]
ca_est_pop = ca_est_pop.rename(columns={'est_total_pop': 'Total Population 2021'})
ca_est_pop = ca_est_pop.rename(columns={'Census_Tract': 'Census Tract'})

In [51]:
merged_df = pd.merge(metric_enviroscreen_data, ca_est_pop[['Census Tract', 'Total Population 2021']], on='Census Tract', how='left')

# Move our merged 2021 pop column towards the front
column_to_move = 'Total Population 2021'
col = merged_df.pop(column_to_move)
merged_df.insert(3, column_to_move, col)


In [52]:
merged_df = merged_df.rename(columns={'Total Population': 'Total Population 2019'})

merged_df

,Census Tract,California County,Total Population 2019,Total Population 2021,Drinking Water,Drinking Water Pctl,Groundwater Threats,Asthma,Low Birth Weight,Cardiovascular Disease,Education,Linguistic Isolation,Poverty,Unemployment,Housing Burden
0,6019001100,Fresno,2780,3166.0,733.946935,84.388660,54.25,129.54,7.80,21.47,44.5,16.0,76.0,12.8,30.3
1,6077000700,San Joaquin,4680,5284.0,389.846569,41.551143,78.60,105.88,6.88,20.26,46.4,29.7,73.2,19.8,31.2
2,6037204920,Los Angeles,2751,2623.0,787.940335,92.531535,20.50,76.10,7.11,20.87,52.2,17.1,62.6,6.4,20.3
3,6019000700,Fresno,3664,NaN,733.946935,84.388660,9.50,139.45,10.65,22.68,41.4,15.7,65.7,15.7,35.4
4,6019000200,Fresno,2689,2861.0,733.946935,84.388660,28.25,139.08,10.25,22.64,43.6,20.0,72.7,13.7,32.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8030,6107004000,Tulare,582,561.0,634.315048,71.150244,0.00,61.64,NaN,21.22,43.6,NaN,79.6,NaN,NaN
8031,6109985202,Tuolumne,2509,2038.0,393.187253,42.725116,3.00,68.79,NaN,22.89,34.1,NaN,NaN,NaN,NaN
8032,6111001206,Ventura,778,NaN,406.952403,44.810791,19.05,48.36,NaN,8.77,NaN,NaN,17.1,NaN,24.4
8033,6111003012,Ventura,675,NaN,663.986626,72.923692,0.30,45.35,NaN,12.25,NaN,51.9,96.7,NaN,NaN


In [55]:
merged_df.columns

Index(['Census Tract', 'California County', 'Total Population 2019',
       'Total Population 2021', 'Drinking Water', 'Drinking Water Pctl',
       'Groundwater Threats', 'Asthma', 'Low Birth Weight',
       'Cardiovascular Disease', 'Education', 'Linguistic Isolation',
       'Poverty', 'Unemployment', 'Housing Burden'],
      dtype='object')

The following metric calculations cannot be used with 2021 population data as their CalEnviroscreen metrics are already in percentages calculated with 2019 data

In [86]:
# Create a directory to store the CSV files if it doesn't exist
output_dir = 'output_csv_files'
os.makedirs(output_dir, exist_ok=True)

# Columns to loop through
columns_to_process = [
    'Low Birth Weight',
    'Cardiovascular Disease',
    'Education',
    'Linguistic Isolation',
    'Poverty',
    'Unemployment',
    'Housing Burden'
]

# Loop through columns to process
for column in columns_to_process:
    # Create new DataFrame
    new_df = merged_df[['Census Tract', 'California County', 'Total Population 2019']].copy()
    
    # Calculate new column name
    new_column_name = column.replace(' ', '_') + '_percent_2019'
    
    # Add new column with the calculated name
    new_df[new_column_name] = merged_df[column]    
    
    # Define CSV file name based on the new column name
    csv_filename = os.path.join(output_dir, new_column_name + '_metric.csv')
    
    # Save the DataFrame to CSV
    new_df.to_csv(csv_filename, index=False)
    
    print(f"Saved DataFrame to: {csv_filename}")
    
    # Output or further process new DataFrame
    display(new_df)

Saved DataFrame to: output_csv_files\Low_Birth_Weight_percent_2019_metric.csv


,Census Tract,California County,Total Population 2019,Low_Birth_Weight_percent_2019
0,6019001100,Fresno,2780,7.80
1,6077000700,San Joaquin,4680,6.88
2,6037204920,Los Angeles,2751,7.11
3,6019000700,Fresno,3664,10.65
4,6019000200,Fresno,2689,10.25
...,...,...,...,...
8030,6107004000,Tulare,582,NaN
8031,6109985202,Tuolumne,2509,NaN
8032,6111001206,Ventura,778,NaN
8033,6111003012,Ventura,675,NaN


Saved DataFrame to: output_csv_files\Cardiovascular_Disease_percent_2019_metric.csv


,Census Tract,California County,Total Population 2019,Cardiovascular_Disease_percent_2019
0,6019001100,Fresno,2780,21.47
1,6077000700,San Joaquin,4680,20.26
2,6037204920,Los Angeles,2751,20.87
3,6019000700,Fresno,3664,22.68
4,6019000200,Fresno,2689,22.64
...,...,...,...,...
8030,6107004000,Tulare,582,21.22
8031,6109985202,Tuolumne,2509,22.89
8032,6111001206,Ventura,778,8.77
8033,6111003012,Ventura,675,12.25


Saved DataFrame to: output_csv_files\Education_percent_2019_metric.csv


...

,Census Tract,California County,Total Population 2019,Poverty_percent_2019
0,6019001100,Fresno,2780,76.0
1,6077000700,San Joaquin,4680,73.2
2,6037204920,Los Angeles,2751,62.6
3,6019000700,Fresno,3664,65.7
4,6019000200,Fresno,2689,72.7
...,...,...,...,...
8030,6107004000,Tulare,582,79.6
8031,6109985202,Tuolumne,2509,NaN
8032,6111001206,Ventura,778,17.1
8033,6111003012,Ventura,675,96.7


Saved DataFrame to: output_csv_files\Unemployment_percent_2019_metric.csv


,Census Tract,California County,Total Population 2019,Unemployment_percent_2019
0,6019001100,Fresno,2780,12.8
1,6077000700,San Joaquin,4680,19.8
2,6037204920,Los Angeles,2751,6.4
3,6019000700,Fresno,3664,15.7
4,6019000200,Fresno,2689,13.7
...,...,...,...,...
8030,6107004000,Tulare,582,NaN
8031,6109985202,Tuolumne,2509,NaN
8032,6111001206,Ventura,778,NaN
8033,6111003012,Ventura,675,NaN


Saved DataFrame to: output_csv_files\Housing_Burden_percent_2019_metric.csv


,Census Tract,California County,Total Population 2019,Housing_Burden_percent_2019
0,6019001100,Fresno,2780,30.3
1,6077000700,San Joaquin,4680,31.2
2,6037204920,Los Angeles,2751,20.3
3,6019000700,Fresno,3664,35.4
4,6019000200,Fresno,2689,32.7
...,...,...,...,...
8030,6107004000,Tulare,582,NaN
8031,6109985202,Tuolumne,2509,NaN
8032,6111001206,Ventura,778,24.4
8033,6111003012,Ventura,675,NaN


Asthma percentage can be calculated with 2019 and 2021 as the CalEnviroscreen values are 'Age-adjusted rate of emergency department visits for asthma'

In [84]:
# Columns to loop through
columns_to_process = [
    'Asthma'
]

# Loop through columns to process
for column in columns_to_process:
    # Create new DataFrame
    new_df = merged_df[['Census Tract', 'California County', 'Total Population 2019', 'Total Population 2021']].copy()
    
    # Calculate new columns
    new_column_name = column.replace(' ', '_') + '_rate_emergency_department_visits'
    new_df[new_column_name] = merged_df[column]
    new_df[new_column_name + '_percentage_2019'] = merged_df[column] / merged_df['Total Population 2019'] * 100
    new_df[new_column_name + '_percentage_2021'] = merged_df[column] / merged_df['Total Population 2021'] * 100
    
    # Define CSV file name based on the new column name
    csv_filename = os.path.join(output_dir, new_column_name + '_metric.csv')
    
    # Save the DataFrame to CSV
    new_df.to_csv(csv_filename, index=False)
    
    print(f"Saved DataFrame to: {csv_filename}")
    
    # Output or further process new DataFrame
    display(new_df)

Saved DataFrame to: output_csv_files\Asthma_rate_emergency_department_visits_metric.csv


,Census Tract,California County,Total Population 2019,Total Population 2021,Asthma_rate_emergency_department_visits,Asthma_rate_emergency_department_visits_percentage_2019,Asthma_rate_emergency_department_visits_percentage_2021
0,6019001100,Fresno,2780,3166.0,129.54,4.659712,4.091598
1,6077000700,San Joaquin,4680,5284.0,105.88,2.262393,2.003785
2,6037204920,Los Angeles,2751,2623.0,76.10,2.766267,2.901258
3,6019000700,Fresno,3664,NaN,139.45,3.805950,NaN
4,6019000200,Fresno,2689,2861.0,139.08,5.172183,4.861237
...,...,...,...,...,...,...,...
8030,6107004000,Tulare,582,561.0,61.64,10.591065,10.987522
8031,6109985202,Tuolumne,2509,2038.0,68.79,2.741730,3.375368
8032,6111001206,Ventura,778,NaN,48.36,6.215938,NaN
8033,6111003012,Ventura,675,NaN,45.35,6.718519,NaN


Drinking water is a bit more challenging. The CalEnviroscreen report states the range for the index is 0-1161 and is calculated with: 

1. Drinking water system boundaries and townships were
downloaded and cleaned.
2. Average concentrations for each contaminant were calculated
and associated with each water system and township.
3. The systems’ and townships’ average water contaminant
concentrations were re-allocated from the associated
boundaries to census tracts. The census tracts were then
ranked to obtain a percentile score for each contaminant and
tract.
4. A census tract contaminant index was calculated as the sum
of the percentiles for all contaminants

California water systems have a high rate of compliance with drinking water standards. In
2017, systems serving an estimated 1.6 percent of the state’s population were in
violation of one or more federal drinking water standards (SWRCB, 2018). The drinking
water contaminant index in CalEnviroScreen 4.0 is not a measure of compliance with
these or California’s state standards. The drinking water contaminant index is a
combination of contaminant data that takes into account the relative concentrations of
different contaminants and whether multiple contaminants are present. The indicator
does not indicate whether water is safe to drink.

There are also a few other potentially useful metrics (all have percentile columns/data):

IMPAIRED WATERBODIES
Contamination of California streams, rivers, lakes, and coastal waters by pollutants can
compromise the use of the water body for drinking, swimming, fishing, aquatic life
protection, and other beneficial uses. When this occurs, such water bodies are considered
“impaired.” Information on impairments to these water bodies can help determine the
extent of environmental degradation within an area.

GROUNDWATER THREATS
Many activities can pose threats to groundwater quality. These include the storage and
disposal of hazardous materials on land and in underground storage tanks at various
types of commercial, industrial, and military sites. Thousands of storage tanks in California
have leaked petroleum or other hazardous substances, degrading soil and groundwater.
Storage tanks are of particular concern when they can affect drinking water supplies. In
addition, the land surrounding these sites may be taken out of service due to perceived
cleanup costs or concerns about liability. Dairy farms and concentrated animal-feeding
operations, which produce large quantities of animal manure pose a threat to
groundwater. Other activities that pose threats to groundwater quality include produced
water ponds, which are generated as a result of oil and gas development. The most
complete sets of information related to sites that may impact groundwater and require
cleanup are maintained by the State Water Resources Control Board. 